In [6]:
import torch
from torchvision import models,transforms,datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import math
import matplotlib.pyplot as plt
import time
import copy
import datetime
import os
from resnet18_dw_diy import resnet18_dw_diy
from resnet18_no_pre_diy import resnet18_no_pre_diy
from train_model import train_model
from count_macs_parm import compute_params_flops

In [7]:
batch_size = 16
pic_size = 224
root = "malevis_train_val_224x224"
num_class = 26
num_epochs = 20

train_transforms = transforms.Compose([transforms.Resize((pic_size, pic_size)),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       ])
val_transforms = transforms.Compose([transforms.Resize((pic_size, pic_size)),
                                     transforms.ToTensor()
                                     ])
train_dataset = datasets.ImageFolder(root = os.path.join(root,"train"),transform = train_transforms)
test_dataset = datasets.ImageFolder(root = os.path.join(root,"val"),transform = val_transforms)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset)
data_loaders = {'train': train_loader, 'val': test_loader}

In [8]:
model = resnet18_no_pre_diy()
compute_params_flops(cnn=model, n_channels=3, n_size=pic_size)
model = resnet18_dw_diy()
compute_params_flops(cnn=model, n_channels=3, n_size=pic_size)




macs, params = 1.824G, 11.190M
macs, params = 346.241M, 1.463M


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1) # Decay LR by a factor of 0.1 every 1 epochs



In [ ]:
print("开始训练！")
train_model(device=device,model=model, pic_size=pic_size,data_loaders=data_loaders,criterion=criterion, optimizer=optimizer, scheduler=exp_lr_scheduler, num_epochs=num_epochs)
